In [1]:
import numpy as np
import pandas as pd

In [2]:
# reading the main data file
file = pd.read_csv("https://raw.githubusercontent.com/jehee1122/MLC-Final_foot-traffic-prediction/main/data/traffic_count.csv")
file.head()

,Unnamed: 0,ID,Segment ID,Roadway Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,...,3:00-4:00PM,4:00-5:00PM,5:00-6:00PM,6:00-7:00PM,7:00-8:00PM,8:00-9:00PM,9:00-10:00PM,10:00-11:00PM,11:00-12:00AM,daily_volume
0,0,1,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/09/2012,20.0,10.0,...,105.0,147.0,120.0,91.0,83.0,74.0,49.0,42.0,42.0,1529.0
1,1,2,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/10/2012,21.0,16.0,...,98.0,133.0,131.0,95.0,73.0,70.0,63.0,42.0,35.0,1424.0
2,2,3,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/11/2012,27.0,14.0,...,115.0,130.0,143.0,106.0,89.0,68.0,64.0,56.0,43.0,1574.0
3,3,4,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/12/2012,22.0,7.0,...,127.0,122.0,144.0,122.0,76.0,64.0,58.0,64.0,43.0,1559.0
4,4,5,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/13/2012,31.0,17.0,...,126.0,133.0,135.0,102.0,106.0,58.0,58.0,55.0,54.0,1659.0


In [3]:
import geopandas as gpd

In [4]:
# reading shapefile. for road segments
geo = gpd.read_file("/Users/smritimohta/Desktop/MLC/Final Project/lion/lion.shp")
geo.head()

,OBJECTID,Street,SAFStreetN,FeatureTyp,SegmentTyp,IncExFlag,RB_Layer,NonPed,TrafDir,TrafSrc,...,FromLeft,ToLeft,FromRight,ToRight,Join_ID,L_PD_Servi,R_PD_Servi,TRUCK_ROUT,SHAPE_Leng,geometry
0,1,EAST 168 STREET,None,0,U,None,B,None,T,DOT,...,599,699,596,716,2251001000000,None,None,None,396.030947,"LINESTRING (1010964.447 241812.261, 1011265.49..."
1,2,WEST 192 STREET,None,0,U,None,B,None,A,DOT,...,58,98,63,99,2798401000000,None,None,None,279.360514,"LINESTRING (1011576.687 255023.583, 1011335.15..."
2,3,UNION AVENUE,None,0,U,None,B,None,W,DOT,...,1017,1079,1016,1084,2728001000000,None,None,None,618.327133,"LINESTRING (1011600.676 239639.743, 1011785.79..."
3,4,UNION AVENUE,BEHAGEN PLAYGROUND COMFORT STA,0,U,None,B,None,W,DOT,...,0,0,0,0,21279502000000X,None,None,None,618.327133,"LINESTRING (1011600.676 239639.743, 1011785.79..."
4,5,UNION AVENUE,BEHAGEN PLAYGROUND FIELD NORTH,0,U,None,B,None,W,DOT,...,0,0,0,0,21279503000000X,None,None,None,618.327133,"LINESTRING (1011600.676 239639.743, 1011785.79..."


In [5]:
# converting segment ID to int
geo['SegmentID'] = geo['SegmentID'].apply(int)

In [6]:
# filtering by required columns
geo1 = geo[['SegmentID','LZip','RZip','LBoro','RBoro','StreetWidt','StreetWi_1','StreetWi_2','BikeLane','TRUCK_ROUT','SHAPE_Leng','geometry']]
geo1.head()

,SegmentID,LZip,RZip,LBoro,RBoro,StreetWidt,StreetWi_1,StreetWi_2,BikeLane,TRUCK_ROUT,SHAPE_Leng,geometry
0,78126,10456,10456,2,2,34.0,34.0,None,None,None,396.030947,"LINESTRING (1010964.447 241812.261, 1011265.49..."
1,79796,10468,10468,2,2,30.0,30.0,None,None,None,279.360514,"LINESTRING (1011576.687 255023.583, 1011335.15..."
2,77356,10459,10459,2,2,34.0,34.0,None,None,None,618.327133,"LINESTRING (1011600.676 239639.743, 1011785.79..."
3,77356,10459,10459,2,2,34.0,34.0,None,None,None,618.327133,"LINESTRING (1011600.676 239639.743, 1011785.79..."
4,77356,10459,10459,2,2,34.0,34.0,None,None,None,618.327133,"LINESTRING (1011600.676 239639.743, 1011785.79..."


In [19]:
#converting crs of zipcode file to 4326 for ease of visualization
geo1['geometry'] = geo1['geometry'].to_crs(epsg=4326)

/Users/smritimohta/opt/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [24]:
# merging both the shapefile and our data on Segment ID
alldata = file.merge(geo1, left_on = 'Segment ID', right_on ='SegmentID')

In [25]:
alldata.head()

,Unnamed: 0,ID,Segment ID,Roadway Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,...,RZip,LBoro,RBoro,StreetWidt,StreetWi_1,StreetWi_2,BikeLane,TRUCK_ROUT,SHAPE_Leng,geometry
0,0,1,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/09/2012,20.0,10.0,...,10304,5,5,36.0,36.0,None,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
1,1,2,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/10/2012,21.0,16.0,...,10304,5,5,36.0,36.0,None,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
2,2,3,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/11/2012,27.0,14.0,...,10304,5,5,36.0,36.0,None,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
3,3,4,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/12/2012,22.0,7.0,...,10304,5,5,36.0,36.0,None,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
4,4,5,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/13/2012,31.0,17.0,...,10304,5,5,36.0,36.0,None,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."


In [13]:
# some basic checks
alldata['StreetWi_2'].isnull().shape

(39575,)

In [15]:
alldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39575 entries, 0 to 39574
Data columns (total 45 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Unnamed: 0     39575 non-null  int64   
 1   ID             39575 non-null  int64   
 2   Segment ID     39575 non-null  int64   
 3   Roadway Name   39575 non-null  object  
 4   From           39575 non-null  object  
 5   To             39575 non-null  object  
 6   Direction      39575 non-null  object  
 7   Date           39575 non-null  object  
 8   12:00-1:00 AM  39574 non-null  float64 
 9   1:00-2:00AM    39574 non-null  float64 
 10  2:00-3:00AM    39574 non-null  float64 
 11  3:00-4:00AM    39574 non-null  float64 
 12  4:00-5:00AM    39574 non-null  float64 
 13  5:00-6:00AM    39574 non-null  float64 
 14  6:00-7:00AM    39574 non-null  float64 
 15  7:00-8:00AM    39574 non-null  float64 
 16  8:00-9:00AM    39574 non-null  float64 
 17  9:00-10:00AM   39574 non-null  

In [26]:
# keeping only required columns in alldata file
alldata.drop(columns=['Unnamed: 0','ID','SegmentID','RZip','RBoro', 'StreetWi_2'], axis=1, inplace=True)
alldata.head()

,Segment ID,Roadway Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,...,11:00-12:00AM,daily_volume,LZip,LBoro,StreetWidt,StreetWi_1,BikeLane,TRUCK_ROUT,SHAPE_Leng,geometry
0,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/09/2012,20.0,10.0,11.0,14.0,...,42.0,1529.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
1,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/10/2012,21.0,16.0,8.0,6.0,...,35.0,1424.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
2,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/11/2012,27.0,14.0,6.0,5.0,...,43.0,1574.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
3,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/12/2012,22.0,7.0,7.0,8.0,...,43.0,1559.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
4,15540,BEACH STREET,UNION PLACE,VAN DUZER STREET,NB,01/13/2012,31.0,17.0,7.0,5.0,...,54.0,1659.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."


In [27]:
# some preprocessing of data
alldata['Roadway Name'] = alldata['Roadway Name'].str.title()
alldata['From'] = alldata['From'].str.title()
alldata['To'] = alldata['To'].str.title()

In [28]:
alldata['Roadway Name'] = alldata['Roadway Name'].str.replace('Street','St')
alldata['Roadway Name'] = alldata['Roadway Name'].str.replace('Avenue','Ave')
alldata['Roadway Name'] = alldata['Roadway Name'].str.replace('Boulevard','Blvd')
alldata['Roadway Name'] = alldata['Roadway Name'].str.replace('West','W')
alldata['Roadway Name'] = alldata['Roadway Name'].str.replace('East','E')
alldata['Roadway Name'] = alldata['Roadway Name'].str.replace('Place','Pl')

In [29]:
alldata['From'] = alldata['From'].str.replace('street','st')
alldata['From'] = alldata['From'].str.replace('avenue','ave')
alldata['From'] = alldata['From'].str.replace('boulevard','blvd')
alldata['From'] = alldata['From'].str.replace('West','W')
alldata['From'] = alldata['From'].str.replace('East','E')
alldata['From'] = alldata['From'].str.replace('Place','Pl')

In [30]:
alldata['To'] = alldata['To'].str.replace('street','st')
alldata['To'] = alldata['To'].str.replace('avenue','ave')
alldata['To'] = alldata['To'].str.replace('boulevard','blvd')
alldata['To'] = alldata['To'].str.replace('West','W')
alldata['To'] = alldata['To'].str.replace('East','E')
alldata['To'] = alldata['To'].str.replace('Place','Pl')

In [31]:
alldata.head()

,Segment ID,Roadway Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,...,11:00-12:00AM,daily_volume,LZip,LBoro,StreetWidt,StreetWi_1,BikeLane,TRUCK_ROUT,SHAPE_Leng,geometry
0,15540,Beach St,Union Pl,Van Duzer Street,NB,01/09/2012,20.0,10.0,11.0,14.0,...,42.0,1529.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
1,15540,Beach St,Union Pl,Van Duzer Street,NB,01/10/2012,21.0,16.0,8.0,6.0,...,35.0,1424.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
2,15540,Beach St,Union Pl,Van Duzer Street,NB,01/11/2012,27.0,14.0,6.0,5.0,...,43.0,1574.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
3,15540,Beach St,Union Pl,Van Duzer Street,NB,01/12/2012,22.0,7.0,7.0,8.0,...,43.0,1559.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."
4,15540,Beach St,Union Pl,Van Duzer Street,NB,01/13/2012,31.0,17.0,7.0,5.0,...,54.0,1659.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6..."


In [32]:
import datetime

In [33]:
# converting date to datetime
alldata['datetime'] = pd.to_datetime(alldata['Date'])

In [36]:
# extracting day of week
alldata['DOW'] = alldata['datetime'].dt.day_name()
alldata.head()

,Segment ID,Roadway Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,...,LZip,LBoro,StreetWidt,StreetWi_1,BikeLane,TRUCK_ROUT,SHAPE_Leng,geometry,datetime,DOW
0,15540,Beach St,Union Pl,Van Duzer Street,NB,01/09/2012,20.0,10.0,11.0,14.0,...,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-09,Monday
1,15540,Beach St,Union Pl,Van Duzer Street,NB,01/10/2012,21.0,16.0,8.0,6.0,...,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-10,Tuesday
2,15540,Beach St,Union Pl,Van Duzer Street,NB,01/11/2012,27.0,14.0,6.0,5.0,...,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-11,Wednesday
3,15540,Beach St,Union Pl,Van Duzer Street,NB,01/12/2012,22.0,7.0,7.0,8.0,...,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-12,Thursday
4,15540,Beach St,Union Pl,Van Duzer Street,NB,01/13/2012,31.0,17.0,7.0,5.0,...,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-13,Friday


In [38]:
alldata[alldata['DOW']=='Monday'].shape

(4440, 41)

In [39]:
alldata[alldata['DOW']=='Tuesday'].shape

(4698, 41)

In [40]:
alldata[alldata['DOW']=='Wednesday'].shape

(4713, 41)

In [41]:
alldata[alldata['DOW']=='Saturday'].shape

(8107, 41)

In [42]:
alldata[alldata['DOW']=='Sunday'].shape

(8103, 41)

In [288]:
pd.set_option('display.max_columns', None)
alldata.head(2)

,Segment ID,Roadway Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,4:00-5:00AM,5:00-6:00AM,6:00-7:00AM,7:00-8:00AM,8:00-9:00AM,9:00-10:00AM,10:00-11:00AM,11:00-12:00PM,12:00-1:00PM,1:00-2:00PM,2:00-3:00PM,3:00-4:00PM,4:00-5:00PM,5:00-6:00PM,6:00-7:00PM,7:00-8:00PM,8:00-9:00PM,9:00-10:00PM,10:00-11:00PM,11:00-12:00AM,daily_volume,LZip,LBoro,StreetWidt,StreetWi_1,BikeLane,TRUCK_ROUT,SHAPE_Leng,geometry,datetime,DOW,day
0,15540,Beach St,Union Pl,Van Duzer Street,NB,01/09/2012,20.0,10.0,11.0,14.0,13.0,20.0,34.0,66.0,100.0,52.0,68.0,85.0,85.0,94.0,104.0,105.0,147.0,120.0,91.0,83.0,74.0,49.0,42.0,42.0,1529.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-09,Monday,Weekday
1,15540,Beach St,Union Pl,Van Duzer Street,NB,01/10/2012,21.0,16.0,8.0,6.0,13.0,13.0,31.0,70.0,67.0,45.0,57.0,67.0,73.0,95.0,102.0,98.0,133.0,131.0,95.0,73.0,70.0,63.0,42.0,35.0,1424.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-10,Tuesday,Weekday


In [49]:
# adding a column to specify weekday/weekend category in each row for further use
day = []
for i in alldata['DOW']:
    if (i == 'Monday') | (i == 'Tuesday') | (i == 'Wednesday') | (i == 'Thursday') | (i == 'Friday'):
        day.append('Weekday')
    else: 
        day.append('Weekend')
        
day

['Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekend',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekend',
 'Weekend',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekend',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekend',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'Weekday',
 'We

In [51]:
alldata['day'] = day

In [289]:
alldata.head(2)

,Segment ID,Roadway Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,4:00-5:00AM,5:00-6:00AM,6:00-7:00AM,7:00-8:00AM,8:00-9:00AM,9:00-10:00AM,10:00-11:00AM,11:00-12:00PM,12:00-1:00PM,1:00-2:00PM,2:00-3:00PM,3:00-4:00PM,4:00-5:00PM,5:00-6:00PM,6:00-7:00PM,7:00-8:00PM,8:00-9:00PM,9:00-10:00PM,10:00-11:00PM,11:00-12:00AM,daily_volume,LZip,LBoro,StreetWidt,StreetWi_1,BikeLane,TRUCK_ROUT,SHAPE_Leng,geometry,datetime,DOW,day
0,15540,Beach St,Union Pl,Van Duzer Street,NB,01/09/2012,20.0,10.0,11.0,14.0,13.0,20.0,34.0,66.0,100.0,52.0,68.0,85.0,85.0,94.0,104.0,105.0,147.0,120.0,91.0,83.0,74.0,49.0,42.0,42.0,1529.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-09,Monday,Weekday
1,15540,Beach St,Union Pl,Van Duzer Street,NB,01/10/2012,21.0,16.0,8.0,6.0,13.0,13.0,31.0,70.0,67.0,45.0,57.0,67.0,73.0,95.0,102.0,98.0,133.0,131.0,95.0,73.0,70.0,63.0,42.0,35.0,1424.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-10,Tuesday,Weekday


In [125]:
# we notice that in our dataset, there are a lot of segments that have small no. of records (outliers)
# checking for record counts for each segment so we can remove them later.
clean = alldata[['Segment ID', 'datetime']].groupby(by=['Segment ID'], as_index=False).count()

In [126]:
clean.sort_values(by='datetime', ascending=False)

,Segment ID,datetime
154,25210,390
1470,176295,264
642,69696,176
1563,251949,144
357,35041,138
...,...,...
438,41761,2
7,1886,2
6,1885,2
5,1884,2


### For id count > = 8

In [127]:
new = clean[clean['datetime']>= 8]

In [128]:
new

,Segment ID,datetime
1,646,8
2,1416,18
3,1421,8
4,1883,20
8,2147,26
...,...,...
1712,9014764,9
1713,9017020,54
1714,9017037,9
1715,9017039,27


In [129]:
new_id = list(new['Segment ID'])

In [133]:
len(new_id)

1687

In [139]:
new_id

[646,
 1416,
 1421,
 1883,
 2147,
 2153,
 2367,
 2369,
 2760,
 2930,
 2933,
 2984,
 3017,
 3115,
 3720,
 4234,
 4237,
 4297,
 4340,
 4832,
 4876,
 5520,
 6644,
 7431,
 7564,
 7886,
 8070,
 8119,
 8152,
 8288,
 8494,
 9661,
 9788,
 9896,
 9922,
 10143,
 10164,
 10188,
 10196,
 10335,
 11273,
 11362,
 11449,
 11599,
 12198,
 12215,
 12806,
 12809,
 12829,
 12832,
 12945,
 13103,
 13120,
 13133,
 14422,
 14602,
 15047,
 15540,
 15736,
 15741,
 17319,
 17414,
 17457,
 17642,
 17652,
 17737,
 18635,
 18641,
 18653,
 18655,
 18852,
 19163,
 19202,
 19244,
 19248,
 19259,
 19313,
 19437,
 19461,
 19463,
 19555,
 19559,
 19566,
 19641,
 19653,
 19761,
 19762,
 19805,
 19813,
 19870,
 19874,
 19978,
 19991,
 20138,
 20147,
 20149,
 20157,
 20278,
 20335,
 20337,
 20340,
 20395,
 20561,
 21024,
 21028,
 21033,
 21110,
 21115,
 21139,
 21142,
 21148,
 21164,
 21327,
 21331,
 21399,
 21490,
 22025,
 22026,
 22033,
 22036,
 22219,
 22221,
 22227,
 22345,
 22379,
 22409,
 22534,
 22539,
 22541,
 228

In [141]:
alldata_new = alldata['Segment ID'].isin(new_id)
alldata_new

0        True
1        True
2        True
3        True
4        True
         ... 
39570    True
39571    True
39572    True
39573    True
39574    True
Name: Segment ID, Length: 39575, dtype: bool

In [137]:
alldata_new8 = alldata[alldata_new]

In [143]:
# data with id records count >= 8
alldata_new8

,Segment ID,Roadway Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,4:00-5:00AM,5:00-6:00AM,6:00-7:00AM,7:00-8:00AM,8:00-9:00AM,9:00-10:00AM,10:00-11:00AM,11:00-12:00PM,12:00-1:00PM,1:00-2:00PM,2:00-3:00PM,3:00-4:00PM,4:00-5:00PM,5:00-6:00PM,6:00-7:00PM,7:00-8:00PM,8:00-9:00PM,9:00-10:00PM,10:00-11:00PM,11:00-12:00AM,daily_volume,LZip,LBoro,StreetWidt,StreetWi_1,BikeLane,TRUCK_ROUT,SHAPE_Leng,geometry,datetime,DOW,day
0,15540,Beach St,Union Pl,Van Duzer Street,NB,01/09/2012,20.0,10.0,11.0,14.0,13.0,20.0,34.0,66.0,100.0,52.0,68.0,85.0,85.0,94.0,104.0,105.0,147.0,120.0,91.0,83.0,74.0,49.0,42.0,42.0,1529.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-09,Monday,Weekday
1,15540,Beach St,Union Pl,Van Duzer Street,NB,01/10/2012,21.0,16.0,8.0,6.0,13.0,13.0,31.0,70.0,67.0,45.0,57.0,67.0,73.0,95.0,102.0,98.0,133.0,131.0,95.0,73.0,70.0,63.0,42.0,35.0,1424.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-10,Tuesday,Weekday
2,15540,Beach St,Union Pl,Van Duzer Street,NB,01/11/2012,27.0,14.0,6.0,5.0,12.0,16.0,34.0,75.0,69.0,71.0,67.0,70.0,90.0,89.0,115.0,115.0,130.0,143.0,106.0,89.0,68.0,64.0,56.0,43.0,1574.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-11,Wednesday,Weekday
3,15540,Beach St,Union Pl,Van Duzer Street,NB,01/12/2012,22.0,7.0,7.0,8.0,11.0,12.0,33.0,75.0,89.0,66.0,70.0,60.0,105.0,103.0,71.0,127.0,122.0,144.0,122.0,76.0,64.0,58.0,64.0,43.0,1559.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-12,Thursday,Weekday
4,15540,Beach St,Union Pl,Van Duzer Street,NB,01/13/2012,31.0,17.0,7.0,5.0,13.0,28.0,29.0,68.0,84.0,64.0,83.0,89.0,88.0,113.0,113.0,126.0,133.0,135.0,102.0,106.0,58.0,58.0,55.0,54.0,1659.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-13,Friday,Weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39570,152046,Woodhaven Blvd,Eliot Ave,Lirr Main Line,NB,10/09/2019,290.0,189.0,144.0,165.0,483.0,1357.0,1741.0,1550.0,1357.0,1914.0,1750.0,1702.0,1541.0,1383.0,1467.0,1460.0,1045.0,993.0,938.0,923.0,834.0,748.0,632.0,342.0,24948.0,11373,4,30.0,68.0,None,2,80.856001,"LINESTRING (-73.87141 40.72952, -73.87137 40.7...",2019-10-09,Wednesday,Weekday
39571,152046,Woodhaven Blvd,Eliot Ave,Lirr Main Line,NB,10/10/2019,203.0,146.0,153.0,229.0,520.0,1124.0,1677.0,1462.0,1554.0,1595.0,1245.0,1194.0,1336.0,1322.0,1250.0,1242.0,1059.0,982.0,810.0,724.0,796.0,741.0,586.0,479.0,22429.0,11373,4,30.0,68.0,None,2,80.856001,"LINESTRING (-73.87141 40.72952, -73.87137 40.7...",2019-10-10,Thursday,Weekday
39572,152046,Woodhaven Blvd,Eliot Ave,Lirr Main Line,NB,10/11/2019,269.0,243.0,166.0,183.0,423.0,1192.0,2145.0,2050.0,2055.0,1896.0,1552.0,1519.0,1483.0,1427.0,1557.0,1677.0,1577.0,1528.0,1541.0,1483.0,1343.0,1146.0,932.0,640.0,30027.0,11373,4,30.0,68.0,None,2,80.856001,"LINESTRING (-73.87141 40.72952, -73.87137 40.7...",2019-10-11,Friday,Weekday
39573,152046,Woodhaven Blvd,Eliot Ave,Lirr Main Line,NB,10/12/2019,537.0,389.0,253.0,212.0,307.0,559.0,894.0,1148.0,1395.0,1530.0,1549.0,1638.0,1605.0,1685.0,1700.0,1700.0,1641.0,1682.0,1599.0,1518.0,1290.0,1097.0,990.0,768.0,27686.0,11373,4,30.0,68.0,None,2,80.856001,"LINESTRING (-73.87141 40.72952, -73.87137 40.7...",2019-10-12,Saturday,Weekend


In [290]:
# converting the resultant dataframe to csv file for usage in hmm
alldata_new8.to_csv("alldata_new8.csv")

In [144]:
# grouping the filtered data (by >= 8 id record counts) by roadway names with direction and weekday/weekend
alldata_grouped8 = alldata_new8[['Segment ID','LZip','LBoro','Roadway Name','From','To','Direction','12:00-1:00 AM', '1:00-2:00AM', '2:00-3:00AM',
       '3:00-4:00AM', '4:00-5:00AM', '5:00-6:00AM', '6:00-7:00AM',
       '7:00-8:00AM', '8:00-9:00AM', '9:00-10:00AM', '10:00-11:00AM',
       '11:00-12:00PM', '12:00-1:00PM', '1:00-2:00PM', '2:00-3:00PM',
       '3:00-4:00PM', '4:00-5:00PM', '5:00-6:00PM', '6:00-7:00PM',
       '7:00-8:00PM', '8:00-9:00PM', '9:00-10:00PM', '10:00-11:00PM',
       '11:00-12:00AM','daily_volume','day']].groupby(by=['Roadway Name','From','To','day','Direction','LBoro','Segment ID','LZip']).mean()


In [145]:
alldata_grouped8 = alldata_grouped8.sort_values(by=['LBoro','Segment ID'])
alldata_grouped8.head(20)

12:00-1:00 AM  \
Roadway Name From            To                    day     Direction LBoro Segment ID LZip                   
State St     Bridge Street   Battery Pl            Weekday NB        1     22995      10004         344.80   
                                                           SB        1     22995      10004         199.50   
                                                   Weekend NB        1     22995      10004         250.75   
                                                           SB        1     22995      10004         162.75   
Broadway     Beaver Street   Lexington Avenue Line Weekday SB        1     23057      10004         105.80   
                                                   Weekend SB        1     23057      10004         112.50   
                             Morris Street         Weekday SB        1     23057      10004         175.00   
                                                   Weekend SB        1     23057      10004         196.50   
Broadway     Beaver Street   Morris Street         Weekday SB        1     23057      10004         548.20   
                                                   Weekend SB        1     23057      10004         511.75   
Trinity Pl   Edger St        Rector Pl             Weekday NB        1     23061      10006         134.00   
                                                   Weekend NB        1     23061      10006         180.00   
Fulton St    8 Avenue Line   8 Avenue Line         Weekday WB        1     23344      10038         164.80   
                                                   Weekend WB        1     23344      10038         185.00   
             Dutch Street    Nassau Street         Weekday WB        1     23344      10038         115.40   
                                                   Weekend WB        1     23344      10038         135.75   
             Nassau Street   Dutch Street          Weekday WB        1     23344      10038          73.00   
                                                   Weekend WB        1     23344      10038         196.75   
Church St    Duane St        Reade St              Weekday NB        1     23396      10013         574.00   
                                                   Weekend NB        1     23396      10013         819.25   

                                                                                             1:00-2:00AM  \
Roadway Name From            To                    day     Direction LBoro Segment ID LZip                 
State St     Bridge Street   Battery Pl            Weekday NB        1     22995      10004   190.900000   
                                                           SB        1     22995      10004   139.300000   
                                                   Weekend NB        1     22995      10004   276.500000   
                                                           SB        1     22995      10004   131.625000   
Broadway     Beaver Street   Lexington Avenue Line Weekday SB        1     23057      10004    77.800000   
                                                   Weekend SB        1     23057      10004   100.000000   
                             Morris Street         Weekday SB        1     23057      10004   122.800000   
                                                   Weekend SB        1     23057      10004   136.000000   
Broadway     Beaver Street   Morris Street         Weekday SB        1     23057      10004   364.000000   
                                                   Weekend SB        1     23057      10004   381.250000   
Trinity Pl   Edger St        Rector Pl             Weekday NB        1     23061      10006    92.333333   
                                                   Weekend NB        1     23061      10006   155.250000   
Fulton St    8 Avenue Line   8 Avenue Line         Weekday WB        1     23344      10038    81.800000   
                                                   W

In [194]:
# grouping the filtered data (by >= 8 id record counts) by segments with direction and weekday/weekend
alldata_segdir = alldata_new8[['Segment ID','LZip','LBoro','Roadway Name','From','To','Direction','12:00-1:00 AM', '1:00-2:00AM', '2:00-3:00AM',
       '3:00-4:00AM', '4:00-5:00AM', '5:00-6:00AM', '6:00-7:00AM',
       '7:00-8:00AM', '8:00-9:00AM', '9:00-10:00AM', '10:00-11:00AM',
       '11:00-12:00PM', '12:00-1:00PM', '1:00-2:00PM', '2:00-3:00PM',
       '3:00-4:00PM', '4:00-5:00PM', '5:00-6:00PM', '6:00-7:00PM',
       '7:00-8:00PM', '8:00-9:00PM', '9:00-10:00PM', '10:00-11:00PM',
       '11:00-12:00AM','daily_volume','day']].groupby(by=['Segment ID','day','Direction']).mean()


In [195]:
alldata_segdir.head()

LBoro  12:00-1:00 AM  1:00-2:00AM  2:00-3:00AM  \
Segment ID day     Direction                                                   
646        Weekday NB           5.0           26.0        24.00        19.75   
                   SB           5.0           30.5        26.75        18.25   
1416       Weekday NB           5.0           43.2        23.00        11.00   
                   SB           5.0           31.6        16.00         9.80   
           Weekend NB           5.0           98.0        56.75        32.75   

                              3:00-4:00AM  4:00-5:00AM  5:00-6:00AM  \
Segment ID day     Direction                                          
646        Weekday NB               16.50        24.75        69.25   
                   SB               16.25        28.00        34.50   
1416       Weekday NB               11.80        32.00        92.20   
                   SB               11.40        29.40        65.80   
           Weekend NB               26.25        23.25        25.25   

                              6:00-7:00AM  7:00-8:00AM  8:00-9:00AM  \
Segment ID day     Direction                                          
646        Weekday NB              118.75       140.75       187.00   
                   SB               75.75       127.75       178.50   
1416       Weekday NB              130.20       245.80       293.00   
                   SB              218.40       411.60       343.00   
           Weekend NB               47.50        76.25       133.25   

                              9:00-10:00AM  10:00-11:00AM  11:00-12:00PM  \
Segment ID day     Direction                                               
646        Weekday NB               219.25         192.00         236.50   
                   SB               203.75         209.75         235.75   
1416       Weekday NB               196.80         230.60         253.60   
                   SB               232.20         238.60         252.00   
           Weekend NB               206.75         280.75         361.50   

                              12:00-1:00PM  1:00-2:00PM  2:00-3:00PM  \
Segment ID day     Direction                                           
646        Weekday NB               242.50       256.50       257.00   
                   SB               258.25       285.75       270.25   
1416       Weekday NB               273.00       299.80       397.80   
                   SB               272.40       290.60       323.80   
           Weekend NB               365.00       370.00       362.75   

                              3:00-4:00PM  4:00-5:00PM  5:00-6:00PM  \
Segment ID day     Direction                                          
646        Weekday NB              276.75       262.00       203.50   
                   SB              311.00       319.50       248.25   
1416       Weekday NB              384.60       396.40       440.80   
                   SB              359.00       354.80       362.20   
           Weekend NB              347.00       347.25       312.00   

                              6:00-7:00PM  7:00-8:00PM  8:00-9:00PM  \
Segment ID day     Direction                                          
646        Weekday NB              163.25       138.00        90.25   
                   SB              202.75       167.25        99.50   
1416       Weekday NB              389.00       320.80       256.40   
                   SB              331.60       264.40       200.00   
           Weekend NB              307.25       246.50       201.00   

                              9:00-10:00PM  10:00-11:00PM  11:00-12:00AM  \
Segment ID day     Direction                                               
646        Weekday NB                 71.5          43.50          36.25   
                   SB                 72.5          42.50          33.50   
1416       Weekday NB                184.4         122.80          83.80   
                   SB                149.6          94.60  

In [196]:
# converting the resultant dataframe to csv file for usage in clustering 
alldata_segdir.to_csv("grouped data_seg_with direction.csv")

In [147]:
# grouping the filtered data (by >= 8 id record counts) by roadway names with no direction and weekday/weekend
alldata_grouped_nd8 = alldata_new8[['Segment ID','LZip','LBoro','Roadway Name','From','To','12:00-1:00 AM', '1:00-2:00AM', '2:00-3:00AM',
       '3:00-4:00AM', '4:00-5:00AM', '5:00-6:00AM', '6:00-7:00AM',
       '7:00-8:00AM', '8:00-9:00AM', '9:00-10:00AM', '10:00-11:00AM',
       '11:00-12:00PM', '12:00-1:00PM', '1:00-2:00PM', '2:00-3:00PM',
       '3:00-4:00PM', '4:00-5:00PM', '5:00-6:00PM', '6:00-7:00PM',
       '7:00-8:00PM', '8:00-9:00PM', '9:00-10:00PM', '10:00-11:00PM',
       '11:00-12:00AM','daily_volume','day']].groupby(by=['Roadway Name','From','To','day','LBoro','Segment ID','LZip'], as_index=False).mean()


In [148]:
alldata_grouped_nd8 = alldata_grouped_nd8.sort_values(by=['LBoro','Segment ID'])
alldata_grouped_nd8.head(20)

,Roadway Name,From,To,day,LBoro,Segment ID,LZip,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,4:00-5:00AM,5:00-6:00AM,6:00-7:00AM,7:00-8:00AM,8:00-9:00AM,9:00-10:00AM,10:00-11:00AM,11:00-12:00PM,12:00-1:00PM,1:00-2:00PM,2:00-3:00PM,3:00-4:00PM,4:00-5:00PM,5:00-6:00PM,6:00-7:00PM,7:00-8:00PM,8:00-9:00PM,9:00-10:00PM,10:00-11:00PM,11:00-12:00AM,daily_volume
3871,State St,Bridge Street,Battery Pl,Weekday,1,22995,10004,272.15,165.100000,103.200000,79.050000,73.700000,121.650000,211.050000,333.650000,439.250000,475.75,421.750,416.650000,409.550000,393.350000,388.9000,409.300000,377.2000,385.90,359.3000,333.1500,309.500000,311.250000,472.350000,385.300000,7648.000000
3872,State St,Bridge Street,Battery Pl,Weekend,1,22995,10004,206.75,204.062500,145.687500,116.125000,93.000000,95.750000,147.562500,174.312500,202.625000,252.25,297.625,328.687500,349.500000,347.875000,351.9375,365.937500,340.9375,344.50,325.8125,275.9375,257.062500,233.000000,212.312500,192.000000,5861.250000
1216,Broadway,Beaver Street,Lexington Avenue Line,Weekday,1,23057,10004,105.80,77.800000,43.200000,43.400000,51.800000,80.800000,110.400000,117.000000,127.200000,125.60,120.800,120.400000,104.600000,105.800000,102.2000,106.800000,94.0000,104.20,107.4000,135.2000,147.400000,134.200000,129.400000,128.000000,2523.400000
1217,Broadway,Beaver Street,Lexington Avenue Line,Weekend,1,23057,10004,112.50,100.000000,85.750000,65.500000,34.500000,43.000000,35.250000,98.000000,98.000000,89.00,116.250,131.500000,100.750000,103.500000,140.5000,122.000000,121.7500,110.00,114.7500,114.5000,153.750000,110.000000,101.000000,74.750000,2376.500000
1218,Broadway,Beaver Street,Morris Street,Weekday,1,23057,10004,175.00,122.800000,97.200000,76.800000,82.600000,122.200000,155.400000,191.800000,247.800000,296.20,249.200,275.400000,296.400000,284.000000,266.0000,269.800000,237.2000,267.40,244.8000,260.6000,255.200000,284.000000,226.400000,217.800000,5202.000000
1219,Broadway,Beaver Street,Morris Street,Weekend,1,23057,10004,196.50,136.000000,118.250000,99.000000,69.250000,81.750000,90.250000,117.250000,142.250000,157.75,165.000,191.750000,197.250000,189.000000,197.7500,198.250000,182.0000,174.25,202.5000,184.5000,174.000000,207.000000,195.250000,180.750000,3847.500000
1310,Broadway,Beaver Street,Morris Street,Weekday,1,23057,10004,548.20,364.000000,244.200000,209.200000,222.400000,308.000000,470.000000,654.800000,806.200000,854.80,848.400,839.600000,838.600000,858.200000,849.2000,872.000000,831.8000,888.00,803.8000,725.6000,705.800000,842.400000,664.400000,620.000000,15869.600000
1311,Broadway,Beaver Street,Morris Street,Weekend,1,23057,10004,511.75,381.250000,338.500000,302.000000,229.000000,215.250000,260.250000,332.500000,508.000000,656.50,663.750,608.750000,781.000000,777.250000,734.2500,686.000000,601.2500,542.00,573.7500,519.0000,536.000000,576.500000,492.000000,421.750000,12248.250000
3956,Trinity Pl,Edger St,Rector Pl,Weekday,1,23061,10006,134.00,92.333333,72.333333,64.166667,54.333333,108.333333,177.166667,231.166667,366.833333,394.00,421.000,448.333333,416.666667,398.166667,409.0000,415.666667,407.0000,395.00,351.0000,305.5000,235.333333,233.333333,208.666667,186.833333,6526.166667
3957,Trinity Pl,Edger St,Rector Pl,Weekend,1,23061,10006,180.00,155.250000,143.750000,121.750000,94.500000,68.000000,77.750000,135.500000,207.000000,263.50,277.500,327.500000,343.000000,333.000000,327.7500,325.000000,281.7500,255.25,246.7500,216.7500,197.750000,177.250000,164.500000,153.500000,5074.250000


In [149]:
alldata_grouped_nd8.shape

(4370, 32)

In [150]:
# converting to csv file for further usage in clustering
alldata_grouped_nd8.to_csv("clean_nodirection8.csv")

In [151]:
# grouping the filtered data (by >= 8 id record counts) by segments and weekday/weekend
alldata_grouped_seg8 = alldata_new8[['Segment ID','LZip','LBoro','Roadway Name','From','To','12:00-1:00 AM', '1:00-2:00AM', '2:00-3:00AM',
       '3:00-4:00AM', '4:00-5:00AM', '5:00-6:00AM', '6:00-7:00AM',
       '7:00-8:00AM', '8:00-9:00AM', '9:00-10:00AM', '10:00-11:00AM',
       '11:00-12:00PM', '12:00-1:00PM', '1:00-2:00PM', '2:00-3:00PM',
       '3:00-4:00PM', '4:00-5:00PM', '5:00-6:00PM', '6:00-7:00PM',
       '7:00-8:00PM', '8:00-9:00PM', '9:00-10:00PM', '10:00-11:00PM',
       '11:00-12:00AM','daily_volume','day']].groupby(by=['Segment ID','day'], as_index=False).mean()


In [152]:
alldata_grouped_seg8 = alldata_grouped_seg8.sort_values(by=['Segment ID'])
alldata_grouped_seg8.head(20)

,Segment ID,day,LBoro,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,4:00-5:00AM,5:00-6:00AM,6:00-7:00AM,7:00-8:00AM,8:00-9:00AM,9:00-10:00AM,10:00-11:00AM,11:00-12:00PM,12:00-1:00PM,1:00-2:00PM,2:00-3:00PM,3:00-4:00PM,4:00-5:00PM,5:00-6:00PM,6:00-7:00PM,7:00-8:00PM,8:00-9:00PM,9:00-10:00PM,10:00-11:00PM,11:00-12:00AM,daily_volume
0,646,Weekday,5.0,28.250000,25.375000,19.000000,16.375000,26.375000,51.875000,97.250000,134.250000,182.750000,211.500000,200.875000,236.125000,250.375000,271.125000,263.625000,293.875000,290.750000,225.875000,183.000000,152.625000,94.875000,72.000000,43.000000,34.875000,3406.000000
1,1416,Weekday,5.0,37.400000,19.500000,10.400000,11.600000,30.700000,79.000000,174.300000,328.700000,318.000000,214.500000,234.600000,252.800000,272.700000,295.200000,360.800000,371.800000,375.600000,401.500000,360.300000,292.600000,228.200000,167.000000,108.700000,74.800000,5020.700000
2,1416,Weekend,5.0,92.125000,56.250000,30.625000,24.625000,21.000000,22.750000,47.625000,78.000000,156.500000,223.125000,295.250000,344.875000,361.250000,364.750000,342.875000,320.250000,311.000000,292.750000,268.750000,223.250000,188.000000,142.375000,119.875000,89.125000,4417.000000
3,1421,Weekday,5.0,53.750000,30.250000,21.750000,19.000000,25.125000,55.000000,151.500000,315.500000,353.375000,213.000000,219.625000,259.250000,287.125000,294.125000,352.125000,400.375000,395.375000,410.375000,366.250000,315.625000,183.625000,140.125000,87.875000,61.750000,5011.875000
4,1883,Weekday,5.0,37.000000,20.916667,12.916667,6.583333,24.333333,66.250000,168.833333,387.833333,306.250000,206.916667,209.333333,223.333333,262.500000,282.166667,362.916667,376.916667,384.333333,375.250000,363.166667,293.250000,227.500000,168.500000,115.250000,86.750000,4969.000000
5,1883,Weekend,5.0,96.875000,64.625000,31.625000,22.000000,14.750000,21.625000,48.125000,98.375000,172.375000,248.625000,290.500000,347.500000,376.125000,340.625000,337.500000,324.875000,308.375000,295.625000,295.625000,247.750000,190.625000,154.250000,109.125000,92.250000,4529.750000
6,2147,Weekday,5.0,52.388889,31.722222,21.333333,17.777778,34.277778,82.111111,208.833333,453.611111,381.777778,257.666667,255.777778,279.888889,301.944444,321.055556,429.277778,458.777778,433.277778,440.000000,417.277778,357.611111,264.888889,174.388889,120.777778,77.222222,5873.666667
7,2147,Weekend,5.0,79.375000,52.750000,29.125000,21.375000,16.250000,27.000000,47.500000,92.375000,152.625000,195.000000,333.250000,371.875000,378.625000,376.500000,336.500000,335.750000,298.875000,304.875000,282.500000,248.375000,199.500000,159.250000,115.750000,88.375000,4543.375000
8,2153,Weekday,5.0,40.277778,26.277778,12.722222,15.722222,33.944444,96.611111,214.944444,382.611111,390.833333,234.888889,214.333333,244.944444,262.388889,286.055556,341.166667,400.555556,390.333333,386.888889,383.888889,326.000000,238.666667,179.611111,130.666667,85.222222,5319.555556
9,2153,Weekend,5.0,118.000000,79.125000,59.750000,35.875000,27.750000,33.250000,53.250000,104.875000,162.750000,236.750000,294.625000,344.375000,384.375000,379.875000,346.000000,361.750000,325.875000,317.500000,307.500000,268.500000,224.250000,168.875000,147.875000,123.750000,4906.500000


In [153]:
alldata_grouped_seg8.shape

(3318, 28)

In [154]:
#converting to csv file for later use
alldata_grouped_seg8.to_csv("clean_seg8.csv")

### For id count >= 10

In [155]:
# repeating the same process for id record counts>= 10
new10 = clean[clean['datetime']>= 10]

In [156]:
new_id10 = list(new10['Segment ID'])

In [158]:
len(new_id10)

1130

In [159]:
alldata_newten = alldata['Segment ID'].isin(new_id10)

In [160]:
alldata_newten

0         True
1         True
2         True
3         True
4         True
         ...  
39570    False
39571    False
39572    False
39573    False
39574    False
Name: Segment ID, Length: 39575, dtype: bool

In [161]:
alldata_new10 = alldata[alldata_newten]
alldata_new10.head()

,Segment ID,Roadway Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,4:00-5:00AM,5:00-6:00AM,6:00-7:00AM,7:00-8:00AM,8:00-9:00AM,9:00-10:00AM,10:00-11:00AM,11:00-12:00PM,12:00-1:00PM,1:00-2:00PM,2:00-3:00PM,3:00-4:00PM,4:00-5:00PM,5:00-6:00PM,6:00-7:00PM,7:00-8:00PM,8:00-9:00PM,9:00-10:00PM,10:00-11:00PM,11:00-12:00AM,daily_volume,LZip,LBoro,StreetWidt,StreetWi_1,BikeLane,TRUCK_ROUT,SHAPE_Leng,geometry,datetime,DOW,day
0,15540,Beach St,Union Pl,Van Duzer Street,NB,01/09/2012,20.0,10.0,11.0,14.0,13.0,20.0,34.0,66.0,100.0,52.0,68.0,85.0,85.0,94.0,104.0,105.0,147.0,120.0,91.0,83.0,74.0,49.0,42.0,42.0,1529.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-09,Monday,Weekday
1,15540,Beach St,Union Pl,Van Duzer Street,NB,01/10/2012,21.0,16.0,8.0,6.0,13.0,13.0,31.0,70.0,67.0,45.0,57.0,67.0,73.0,95.0,102.0,98.0,133.0,131.0,95.0,73.0,70.0,63.0,42.0,35.0,1424.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-10,Tuesday,Weekday
2,15540,Beach St,Union Pl,Van Duzer Street,NB,01/11/2012,27.0,14.0,6.0,5.0,12.0,16.0,34.0,75.0,69.0,71.0,67.0,70.0,90.0,89.0,115.0,115.0,130.0,143.0,106.0,89.0,68.0,64.0,56.0,43.0,1574.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-11,Wednesday,Weekday
3,15540,Beach St,Union Pl,Van Duzer Street,NB,01/12/2012,22.0,7.0,7.0,8.0,11.0,12.0,33.0,75.0,89.0,66.0,70.0,60.0,105.0,103.0,71.0,127.0,122.0,144.0,122.0,76.0,64.0,58.0,64.0,43.0,1559.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-12,Thursday,Weekday
4,15540,Beach St,Union Pl,Van Duzer Street,NB,01/13/2012,31.0,17.0,7.0,5.0,13.0,28.0,29.0,68.0,84.0,64.0,83.0,89.0,88.0,113.0,113.0,126.0,133.0,135.0,102.0,106.0,58.0,58.0,55.0,54.0,1659.0,10304,5,36.0,36.0,None,None,754.271243,"LINESTRING (-74.07752 40.62774, -74.07972 40.6...",2012-01-13,Friday,Weekday


In [162]:
# grouping the filtered data (by >= 10 id record counts) by roadways,direction and weekday/weekend
alldata_grouped10 = alldata_new10[['Segment ID','LZip','LBoro','Roadway Name','From','To','Direction','12:00-1:00 AM', '1:00-2:00AM', '2:00-3:00AM',
       '3:00-4:00AM', '4:00-5:00AM', '5:00-6:00AM', '6:00-7:00AM',
       '7:00-8:00AM', '8:00-9:00AM', '9:00-10:00AM', '10:00-11:00AM',
       '11:00-12:00PM', '12:00-1:00PM', '1:00-2:00PM', '2:00-3:00PM',
       '3:00-4:00PM', '4:00-5:00PM', '5:00-6:00PM', '6:00-7:00PM',
       '7:00-8:00PM', '8:00-9:00PM', '9:00-10:00PM', '10:00-11:00PM',
       '11:00-12:00AM','daily_volume','day']].groupby(by=['Roadway Name','From','To','day','Direction','LBoro','Segment ID','LZip']).mean()


In [163]:
alldata_grouped10 = alldata_grouped10.sort_values(by=['LBoro','Segment ID'])
alldata_grouped10.head()

12:00-1:00 AM  \
Roadway Name From          To                    day     Direction LBoro Segment ID LZip                   
State St     Bridge Street Battery Pl            Weekday NB        1     22995      10004         344.80   
                                                         SB        1     22995      10004         199.50   
                                                 Weekend NB        1     22995      10004         250.75   
                                                         SB        1     22995      10004         162.75   
Broadway     Beaver Street Lexington Avenue Line Weekday SB        1     23057      10004         105.80   

                                                                                           1:00-2:00AM  \
Roadway Name From          To                    day     Direction LBoro Segment ID LZip                 
State St     Bridge Street Battery Pl            Weekday NB        1     22995      10004      190.900   
                                                         SB        1     22995      10004      139.300   
                                                 Weekend NB        1     22995      10004      276.500   
                                                         SB        1     22995      10004      131.625   
Broadway     Beaver Street Lexington Avenue Line Weekday SB        1     23057      10004       77.800   

                                                                                           2:00-3:00AM  \
Roadway Name From          To                    day     Direction LBoro Segment ID LZip                 
State St     Bridge Street Battery Pl            Weekday NB        1     22995      10004      127.200   
                                                         SB        1     22995      10004       79.200   
                                                 Weekend NB        1     22995      10004      195.875   
                                                         SB        1     22995      10004       95.500   
Broadway     Beaver Street Lexington Avenue Line Weekday SB        1     23057      10004       43.200   

                                                                                           3:00-4:00AM  \
Roadway Name From          To                    day     Direction LBoro Segment ID LZip                 
State St     Bridge Street Battery Pl            Weekday NB        1     22995      10004       87.200   
                                                         SB        1     22995      10004       70.900   
                                                 Weekend NB        1     22995      10004      158.375   
                                                         SB        1     22995      10004       73.875   
Broadway     Beaver Street Lexington Avenue Line Weekday SB        1     23057      10004       43.400   

                                                                                           4:00-5:00AM  \
Roadway Name From          To                    day     Direction LBoro Segment ID LZip                 
State St     Bridge Street Battery Pl            Weekday NB        1     22995      10004       88.400   
                                                         SB        1     22995      10004       59.000   
                                                 Weekend NB        1     22995      10004      114.875   
                                                         SB        1     22995      10004       71.125   
Broadway     Beaver Street Lexington Avenue Line Weekday SB        1     23057      10004       51.800   

                                                                                           5:00-6:00AM  \
Roadway Name From          To                    day     Direction LBoro Segment ID LZip                 
State St     Bridge Street Battery Pl            Weekday NB        1     22995      10004       138.80   
                                            

In [164]:
# grouping the filtered data (by >= 10 id record counts) by roadways, no direction and weekday/weekend
alldata_grouped_nd10 = alldata_new10[['Segment ID','LZip','LBoro','Roadway Name','From','To','12:00-1:00 AM', '1:00-2:00AM', '2:00-3:00AM',
       '3:00-4:00AM', '4:00-5:00AM', '5:00-6:00AM', '6:00-7:00AM',
       '7:00-8:00AM', '8:00-9:00AM', '9:00-10:00AM', '10:00-11:00AM',
       '11:00-12:00PM', '12:00-1:00PM', '1:00-2:00PM', '2:00-3:00PM',
       '3:00-4:00PM', '4:00-5:00PM', '5:00-6:00PM', '6:00-7:00PM',
       '7:00-8:00PM', '8:00-9:00PM', '9:00-10:00PM', '10:00-11:00PM',
       '11:00-12:00AM','daily_volume','day']].groupby(by=['Roadway Name','From','To','day','LBoro','Segment ID','LZip'], as_index=False).mean()


In [165]:
alldata_grouped_nd10 = alldata_grouped_nd10.sort_values(by=['LBoro','Segment ID'])
alldata_grouped_nd10.head()

,Roadway Name,From,To,day,LBoro,Segment ID,LZip,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,4:00-5:00AM,5:00-6:00AM,6:00-7:00AM,7:00-8:00AM,8:00-9:00AM,9:00-10:00AM,10:00-11:00AM,11:00-12:00PM,12:00-1:00PM,1:00-2:00PM,2:00-3:00PM,3:00-4:00PM,4:00-5:00PM,5:00-6:00PM,6:00-7:00PM,7:00-8:00PM,8:00-9:00PM,9:00-10:00PM,10:00-11:00PM,11:00-12:00AM,daily_volume
2917,State St,Bridge Street,Battery Pl,Weekday,1,22995,10004,272.15,165.1000,103.2000,79.050,73.7,121.65,211.0500,333.6500,439.250,475.75,421.750,416.6500,409.55,393.350,388.9000,409.3000,377.2000,385.9,359.3000,333.1500,309.5000,311.25,472.3500,385.30,7648.00
2918,State St,Bridge Street,Battery Pl,Weekend,1,22995,10004,206.75,204.0625,145.6875,116.125,93.0,95.75,147.5625,174.3125,202.625,252.25,297.625,328.6875,349.50,347.875,351.9375,365.9375,340.9375,344.5,325.8125,275.9375,257.0625,233.00,212.3125,192.00,5861.25
1032,Broadway,Beaver Street,Lexington Avenue Line,Weekday,1,23057,10004,105.80,77.8000,43.2000,43.400,51.8,80.80,110.4000,117.0000,127.200,125.60,120.800,120.4000,104.60,105.800,102.2000,106.8000,94.0000,104.2,107.4000,135.2000,147.4000,134.20,129.4000,128.00,2523.40
1033,Broadway,Beaver Street,Lexington Avenue Line,Weekend,1,23057,10004,112.50,100.0000,85.7500,65.500,34.5,43.00,35.2500,98.0000,98.000,89.00,116.250,131.5000,100.75,103.500,140.5000,122.0000,121.7500,110.0,114.7500,114.5000,153.7500,110.00,101.0000,74.75,2376.50
1034,Broadway,Beaver Street,Morris Street,Weekday,1,23057,10004,175.00,122.8000,97.2000,76.800,82.6,122.20,155.4000,191.8000,247.800,296.20,249.200,275.4000,296.40,284.000,266.0000,269.8000,237.2000,267.4,244.8000,260.6000,255.2000,284.00,226.4000,217.80,5202.00


In [166]:
# saving as csv for further use
alldata_grouped_nd10.to_csv("clean_nodirection10.csv")

In [167]:
# grouping the filtered data (by >= 10 id record counts) by segments and weekday/weekend
alldata_grouped_seg10 = alldata_new10[['Segment ID','LZip','LBoro','Roadway Name','From','To','12:00-1:00 AM', '1:00-2:00AM', '2:00-3:00AM',
       '3:00-4:00AM', '4:00-5:00AM', '5:00-6:00AM', '6:00-7:00AM',
       '7:00-8:00AM', '8:00-9:00AM', '9:00-10:00AM', '10:00-11:00AM',
       '11:00-12:00PM', '12:00-1:00PM', '1:00-2:00PM', '2:00-3:00PM',
       '3:00-4:00PM', '4:00-5:00PM', '5:00-6:00PM', '6:00-7:00PM',
       '7:00-8:00PM', '8:00-9:00PM', '9:00-10:00PM', '10:00-11:00PM',
       '11:00-12:00AM','daily_volume','day']].groupby(by=['Segment ID','day'], as_index=False).mean()


In [168]:
alldata_grouped_seg10 = alldata_grouped_seg10.sort_values(by=['Segment ID'])
alldata_grouped_seg10.head()

,Segment ID,day,LBoro,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,4:00-5:00AM,5:00-6:00AM,6:00-7:00AM,7:00-8:00AM,8:00-9:00AM,9:00-10:00AM,10:00-11:00AM,11:00-12:00PM,12:00-1:00PM,1:00-2:00PM,2:00-3:00PM,3:00-4:00PM,4:00-5:00PM,5:00-6:00PM,6:00-7:00PM,7:00-8:00PM,8:00-9:00PM,9:00-10:00PM,10:00-11:00PM,11:00-12:00AM,daily_volume
0,1416,Weekday,5.0,37.400000,19.500000,10.400000,11.600000,30.700000,79.000000,174.300000,328.700000,318.000000,214.500000,234.600000,252.800000,272.700000,295.200000,360.800000,371.800000,375.600000,401.500,360.300000,292.600000,228.200000,167.000000,108.700000,74.800000,5020.700000
1,1416,Weekend,5.0,92.125000,56.250000,30.625000,24.625000,21.000000,22.750000,47.625000,78.000000,156.500000,223.125000,295.250000,344.875000,361.250000,364.750000,342.875000,320.250000,311.000000,292.750,268.750000,223.250000,188.000000,142.375000,119.875000,89.125000,4417.000000
2,1883,Weekday,5.0,37.000000,20.916667,12.916667,6.583333,24.333333,66.250000,168.833333,387.833333,306.250000,206.916667,209.333333,223.333333,262.500000,282.166667,362.916667,376.916667,384.333333,375.250,363.166667,293.250000,227.500000,168.500000,115.250000,86.750000,4969.000000
3,1883,Weekend,5.0,96.875000,64.625000,31.625000,22.000000,14.750000,21.625000,48.125000,98.375000,172.375000,248.625000,290.500000,347.500000,376.125000,340.625000,337.500000,324.875000,308.375000,295.625,295.625000,247.750000,190.625000,154.250000,109.125000,92.250000,4529.750000
4,2147,Weekday,5.0,52.388889,31.722222,21.333333,17.777778,34.277778,82.111111,208.833333,453.611111,381.777778,257.666667,255.777778,279.888889,301.944444,321.055556,429.277778,458.777778,433.277778,440.000,417.277778,357.611111,264.888889,174.388889,120.777778,77.222222,5873.666667


In [169]:
# saving as csv for further use
alldata_grouped_seg10.to_csv("clean_seg10.csv")

In [170]:
alldata_grouped_seg10.shape

(2238, 28)